## 쿨루프 시공 대상 여부 분류
- Tasks
    1. Data Preprocessing
        - 모델 사용을 위해 데이터를 일관성 있게 정리해야 합니다.
    2. Object Detection
        - 전처리 된 데이터를 이용하여 학습을 진행하세요.
        - 인공위성 지도를 이용하여 이미지를 **추가적으로** 10장 수집하고 추론 과정에서 사용하세요.

In [1]:
!pip install keras --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 14.4 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.2.1 which is incompatible.


In [2]:
import os
os.environ['KERAS_BACKEND'] = 'tensorflow'

In [3]:
import gdown, zipfile
import os, glob, shutil
from tqdm import tqdm

import numpy as np
import matplotlib.pyplot as plt

### 데이터셋 다운로드 및 압축 해제
- cool_roof_image.zip : 이미지 데이터 압축 파일
- cool_roof_yolo_labels.zip : YOLO 모델 형식에 맞게 가공된 레이블 파일

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
def dataset_extract(file_name):
    with zipfile.ZipFile(file_name, 'r') as zip_ref:
        file_list = zip_ref.namelist()
        if os.path.exists(f'/content/{file_name[-14:-4]}/'):
            print(f'데이터셋 폴더가 이미 존재합니다.')
            return
        else:
            for f in tqdm(file_list, desc='Extracting', unit='files'):
                zip_ref.extract(member=f, path=f'/content/{file_name[-14:-4]}/')

In [6]:
# img download
def download_file(file_id, save_path) :
    if os.path.exists(save_path) :
        print(f'{save_path} 파일이 이미 존재합니다.')
        return

    gdown.download(id=file_id, output=save_path, quiet=False)

img_file_id = '1MQFnhqGq2JZ6Vpt4Qd-mqdB_nRiEdTDR'
download_file(img_file_id, 'cool_roof_yolo_images_real.zip')

Downloading...
From (original): https://drive.google.com/uc?id=1MQFnhqGq2JZ6Vpt4Qd-mqdB_nRiEdTDR
From (redirected): https://drive.google.com/uc?id=1MQFnhqGq2JZ6Vpt4Qd-mqdB_nRiEdTDR&confirm=t&uuid=ab50fe9f-a654-43d8-881e-1567f45cdca6
To: /content/cool_roof_yolo_images_real.zip
100%|██████████| 239M/239M [00:01<00:00, 221MB/s]


In [7]:
dataset_extract('/content/drive/MyDrive/AIVLE_SCHOOL_5th/미프4차/cool_roof_yolo_labels (1).zip')
dataset_extract('/content/cool_roof_yolo_images_real.zip')
dataset_extract('/content/drive/MyDrive/AIVLE_SCHOOL_5th/미프4차/201~225 labels.zip')

Extracting: 100%|██████████| 25/25 [00:00<00:00, 69.73files/s]


In [8]:
# 잘못된 데이터셋 제외
img_path = '/content/mages_real'
img_files = os.listdir(img_path)

# roof (201) ~ roof (225)
# roof (268) ~ roof (300)

for i in range(201, 226):
    # file_path = '/content/mages_real/'
    file_path_txt = '/content/labels (1)/cool_roof_yolo_labels/obj_train_data/'
    # rm_names = 'roof (' + str(i) + ').jpg'
    rm_txt = 'roof (' + str(i) + ').txt'
    # real_name = file_path + rm_names
    real_txt = file_path_txt + rm_txt
    # os.remove(real_name)
    os.remove(real_txt)

In [9]:
for i in range(268, 301):
    file_path = '/content/mages_real/'
    file_path_txt = '/content/labels (1)/cool_roof_yolo_labels/obj_train_data/'
    rm_names = 'roof (' + str(i) + ').jpg'
    rm_txt = 'roof (' + str(i) + ').txt'
    real_name = file_path + rm_names
    real_txt = file_path_txt + rm_txt
    os.remove(real_name)
    os.remove(real_txt)
    print(real_name, real_txt)

/content/mages_real/roof (268).jpg /content/labels (1)/cool_roof_yolo_labels/obj_train_data/roof (268).txt
/content/mages_real/roof (269).jpg /content/labels (1)/cool_roof_yolo_labels/obj_train_data/roof (269).txt
/content/mages_real/roof (270).jpg /content/labels (1)/cool_roof_yolo_labels/obj_train_data/roof (270).txt
/content/mages_real/roof (271).jpg /content/labels (1)/cool_roof_yolo_labels/obj_train_data/roof (271).txt
/content/mages_real/roof (272).jpg /content/labels (1)/cool_roof_yolo_labels/obj_train_data/roof (272).txt
/content/mages_real/roof (273).jpg /content/labels (1)/cool_roof_yolo_labels/obj_train_data/roof (273).txt
/content/mages_real/roof (274).jpg /content/labels (1)/cool_roof_yolo_labels/obj_train_data/roof (274).txt
/content/mages_real/roof (275).jpg /content/labels (1)/cool_roof_yolo_labels/obj_train_data/roof (275).txt
/content/mages_real/roof (276).jpg /content/labels (1)/cool_roof_yolo_labels/obj_train_data/roof (276).txt
/content/mages_real/roof (277).jpg /c

In [10]:
# 수정된 201~205.txt 라벨 copy
import shutil

# 파일들을 폴더로 복사
# img 복사

new_path = '/content/225 labels'

extension_txt = '.txt'
dest_path_label = '/content/labels (1)/cool_roof_yolo_labels/obj_train_data'
i=0
for data in os.listdir(new_path):
    tmp_path = '/content/225 labels/'
    src_path = tmp_path + data
    # real_img = real_path + data + extension_img
    # real_txt = real_path_label + data + extension_txt
    # dest_img = '/content/data/train/images'
    # dest_txt = '/content/data/train/labels'

    shutil.copy(src_path, dest_path_label)


### 폴더 생성
- YOLO 모델에서 요구하는 폴더의 형식이 있습니다.
- 해당 형식에 맞춰 폴더를 만드세요.

In [11]:
import os

def createFolder(dir):
    try:
        if not os.path.exists(dir):
            os.makedirs(dir)
    except OSError:
        print ('Error: Creating directory. ' +  dir)

createFolder('/content/data')

In [12]:
createFolder('/content/data/train')
createFolder('/content/data/val')

createFolder('/content/data/train/images')
createFolder('/content/data/train/labels')

createFolder('/content/data/val/images')
createFolder('/content/data/val/labels')

In [13]:
import glob

In [14]:
img_path = '/content/mages_real'
img_files = os.listdir(img_path)
data_names = []
for img_file in img_files:
    name = os.path.splitext(img_file)[0]
    data_names.append(name)

In [15]:
len(data_names)

385

### 데이터 스플릿 & 파일 이동
1. 재현을 위한 난수 고정 : 2024
2. 테스트셋 데이터 20%
- 위 설정에 맞게 데이터를 나누고, 위 과정에서 생성한 폴더에 이동시키세요.

In [16]:
import random

idx_list = [x for x in range(len(data_names))]
random.seed(2024)
random.shuffle(idx_list)
print(idx_list)

[177, 81, 333, 302, 144, 323, 254, 375, 49, 346, 261, 133, 363, 1, 21, 228, 10, 301, 339, 4, 249, 58, 146, 19, 95, 11, 221, 319, 256, 356, 343, 267, 381, 200, 156, 99, 138, 294, 331, 235, 5, 273, 282, 62, 98, 262, 8, 298, 270, 94, 300, 56, 194, 16, 286, 180, 227, 136, 160, 243, 43, 61, 18, 39, 59, 192, 123, 281, 131, 96, 66, 275, 329, 364, 185, 271, 14, 203, 220, 355, 35, 259, 70, 327, 82, 248, 190, 151, 52, 73, 68, 202, 25, 292, 153, 305, 257, 97, 330, 216, 106, 164, 89, 266, 76, 168, 283, 13, 74, 260, 189, 205, 309, 137, 197, 284, 348, 351, 147, 113, 341, 313, 174, 312, 191, 84, 287, 24, 214, 353, 383, 173, 217, 344, 186, 83, 373, 354, 119, 92, 182, 126, 371, 78, 303, 6, 244, 184, 17, 358, 293, 170, 40, 65, 361, 87, 130, 295, 377, 110, 12, 140, 127, 141, 201, 380, 54, 328, 196, 245, 349, 338, 85, 30, 3, 175, 195, 91, 45, 299, 101, 79, 316, 384, 34, 324, 225, 107, 264, 150, 279, 55, 183, 112, 211, 116, 229, 274, 219, 121, 47, 318, 48, 165, 206, 148, 311, 357, 234, 306, 159, 250, 233, 

In [17]:
train_num = int(len(data_names) * 0.8)
train_set = data_names[:train_num]

val_set = data_names[train_num:]

In [18]:
train_num, len(val_set)

(308, 77)

In [19]:
import shutil

# 파일들을 폴더로 복사
# img 복사

img_train_path = '/content/data/train/images'

extension_img = '.jpg'
extension_txt = '.txt'
real_path = '/content/mages_real/'
real_path_label = '/content/labels (1)/cool_roof_yolo_labels/obj_train_data/'

for data in train_set:
    real_img = real_path + data + extension_img
    real_txt = real_path_label + data + extension_txt
    dest_img = '/content/data/train/images'
    dest_txt = '/content/data/train/labels'

    shutil.copy(real_img, dest_img)
    shutil.copy(real_txt, dest_txt)


In [20]:
extension_img = '.jpg'
extension_txt = '.txt'
real_path = '/content/mages_real/'
real_path_label = '/content/labels (1)/cool_roof_yolo_labels/obj_train_data/'

for data in val_set:
    real_img = real_path + data + extension_img
    real_txt = real_path_label + data + extension_txt
    dest_img = '/content/data/val/images'
    dest_txt = '/content/data/val/labels'

    shutil.copy(real_img, dest_img)
    shutil.copy(real_txt, dest_txt)

In [21]:
img_list = os.listdir('/content/data/val/images')
print(len(img_list))

77


In [22]:
img_list = os.listdir('/content/data/train/images')
print(len(img_list))

308


### YOLO 모델에 적용할 YAML 생성하기
- 지붕에는 두 가지 형태가 있습니다. 클래스 구분에 주의하세요.
- cool roof
- generic roof

In [23]:
import yaml

data = {
    'train': '/content/data/train',
    'val': '/content/data/val',
    # 'test': '/content/data/test',
    'nc': 2,
    'names': {
        1: 'generic roof',
        0: 'cool roof'
    }
}

with open('./data.yaml', 'w') as f:
    yaml.dump(data, f)

print("data.yaml 파일이 생성되었습니다.")

data.yaml 파일이 생성되었습니다.


In [24]:
with open('./data.yaml', 'r') as f:
    lines = yaml.safe_load(f)
    print(lines)

{'names': {0: 'cool roof', 1: 'generic roof'}, 'nc': 2, 'train': '/content/data/train', 'val': '/content/data/val'}


### YOLO v8 모델
- yaml 파일의 경로 설정에 주의하세요.

In [25]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.4/750.4 kB 7.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [26]:
from ultralytics import YOLO, settings
settings['datasets_dir'] = '/content/data'
settings.update()
settings

{'settings_version': '0.0.4',
 'datasets_dir': '/content/data',
 'weights_dir': 'weights',
 'runs_dir': 'runs',
 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb',
 'sync': True,
 'api_key': '',
 'openai_api_key': '',
 'clearml': True,
 'comet': True,
 'dvc': True,
 'hub': True,
 'mlflow': True,
 'neptune': True,
 'raytune': True,
 'tensorboard': True,
 'wandb': True}

In [32]:
model = YOLO(model='yolov8s.yaml', task='detect')

In [33]:
model.train(data='data.yaml',
            epochs=150,
            patience=5,
            pretrained=True,
            verbose=True,
            seed=2024,
            )

Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.yaml, data=data.yaml, epochs=150, time=None, patience=5, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=2024, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, 

train: Scanning /content/data/train/labels.cache... 308 images, 0 backgrounds, 0 corrupt: 100%|██████████| 308/308 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))



/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
val: Scanning /content/data/val/labels.cache... 77 images, 0 backgrounds, 0 corrupt: 100%|██████████| 77/77 [00:00<?, ?it/s]


Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 2 dataloader workers
Logging results to runs/detect/train4
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all         77        603      0.637      0.689      0.678      0.387
EarlyStopping: Training stopped early as no improvement observed in last 5 epochs. Best results observed at epoch 57, best model saved as best.pt.
To update EarlyStopping(patience=5) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



62 epochs completed in 0.247 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 22.5MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics YOLOv8.1.47 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLOv8s summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:01<00:00,  1.61it/s]


                   all         77        603      0.699      0.699      0.731      0.445
             cool roof         77        177      0.656      0.689      0.684      0.424
          generic roof         77        426      0.743      0.709      0.777      0.467
Speed: 0.3ms preprocess, 4.6ms inference, 0.0ms loss, 3.5ms postprocess per image
Results saved to runs/detect/train4


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7a67f34e6560>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04804

In [34]:
# test set
dataset_extract('/content/drive/MyDrive/AIVLE_SCHOOL_5th/미프4차/test.zip')

데이터셋 폴더가 이미 존재합니다.


In [35]:
results = model.predict(source='/content/프4차/test/*',
                        iou=0.6,
                        conf=0.5,
                        save=True,
                        line_width=2)


image 1/21 /content/프4차/test/test10.jpg: 512x640 1 cool roof, 4 generic roofs, 74.3ms
image 2/21 /content/프4차/test/test11.jpg: 608x640 13 cool roofs, 22 generic roofs, 73.1ms
image 3/21 /content/프4차/test/test12.jpg: 608x640 4 cool roofs, 11 generic roofs, 16.4ms
image 4/21 /content/프4차/test/test13.jpg: 640x640 2 cool roofs, 19 generic roofs, 17.4ms
image 5/21 /content/프4차/test/test14.jpg: 576x640 3 cool roofs, 14 generic roofs, 71.8ms
image 6/21 /content/프4차/test/test15.jpg: 608x640 3 cool roofs, 16 generic roofs, 17.3ms
image 7/21 /content/프4차/test/test16.jpg: 640x640 6 cool roofs, 25 generic roofs, 17.5ms
image 8/21 /content/프4차/test/test17.jpg: 640x448 3 cool roofs, 16 generic roofs, 78.9ms
image 9/21 /content/프4차/test/test18.jpg: 544x640 1 cool roof, 37 generic roofs, 77.1ms
image 10/21 /content/프4차/test/test19.jpg: 640x640 1 cool roof, 11 generic roofs, 17.6ms
image 11/21 /content/프4차/test/test2.jpg: 576x640 (no detections), 17.2ms
image 12/21 /content/프4차

In [33]:
# for r in results :
#     r_bbox = r.boxes
#     print(r_bbox)

In [34]:
import cv2
import os

# 바운딩 박스 정보가 있는 경로
bbox_folder_path = '/content/olo_labels/obj_train_data'

# 이미지가 있는 경로
image_folder_paths = ['/content/data/train/images', '/content/data/val/images']

# 클래스 번호에 대응되는 색깔 (예시)
class_colors = {
    0: (255, 0, 0),   # 클래스 0의 바운딩 박스 색깔 (파랑)
    1: (0, 255, 0),   # 클래스 1의 바운딩 박스 색깔 (초록)
    # 클래스별로 추가할 수 있음
}

# 결과를 저장할 폴더 경로
result_folder_path = '/content/data/images/annotated'

# 결과를 저장할 폴더 생성
os.makedirs(result_folder_path, exist_ok=True)

# 이미지 경로에 대해 반복
for image_folder_path in image_folder_paths:
    # 이미지 파일들의 경로 가져오기
    image_files = os.listdir(image_folder_path)

    # 각 이미지 파일에 대해 처리
    for image_file in image_files:
        # 이미지 파일 경로
        image_path = os.path.join(image_folder_path, image_file)

        # 바운딩 박스 파일 경로
        bbox_file_path = os.path.join(bbox_folder_path, image_file.replace('.jpg', '.txt'))

        # 이미지 읽기
        image = cv2.imread(image_path)

        # 이미지 크기 가져오기
        height, width = image.shape[:2]

        # 바운딩 박스 정보 읽기
        if os.path.exists(bbox_file_path):
            with open(bbox_file_path, 'r') as f:
                lines = f.readlines()

            # 각 바운딩 박스에 대해 이미지에 그리기
            for line in lines:
                class_id, center_x, center_y, bbox_width, bbox_height = map(float, line.strip().split())

                # 바운딩 박스의 상대적인 좌표와 크기를 절대 좌표로 변환
                x = int((center_x - bbox_width / 2) * width)
                y = int((center_y - bbox_height / 2) * height)
                w = int(bbox_width * width)
                h = int(bbox_height * height)

                # 클래스에 따른 색상 가져오기
                color = class_colors.get(int(class_id), (0, 0, 255))  # 클래스가 없는 경우 빨강색 사용

                # 이미지에 바운딩 박스 그리기
                cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)

            # 결과 저장할 경로 설정
            output_path = os.path.join(result_folder_path, image_file)

            # 이미지에 바운딩 박스가 그려진 결과 저장
            cv2.imwrite(output_path, image)